# Predicting failing cable joints with a classification model

## Introduction
Our powergrid is an intricate network of cables/lines and stations that transport electricity from powerplants, solar parcs, wind parcs, etc. to e.g. your home. Within this network the cables are connected using cable joints (or 'Moffen' in Dutch). A range of joint types are used in our grid as these have improved over time. Older joints might experience failure due to a range of conditions. One of Alliander's main objectives is to have a reliable grid. Therefore, it is important to know which joint types are prone to failure in order to prevent power failures. 

Today, Alliander is going to ask you to come up with a way to predict cable joint failures using classification models. We know that joints fail due to large temperature variations and subsequently cause short circuits. The failure of cable joints is difficult to determine. However, we know that there is a relationship between a joint failure and the depth of a joint, the joint type, soil type and groundwater level. We also know that joint type is the most dominant factor to joint failures.

Using the information supplied above and the supplied dataset containing information on the cable joints present in our grid, try to come up with classification models that predict the failure of cable joints.

**Contents:**
1. [Import packages and load the data](#1)
1. [Data Exploration](#2)
1. [Preparation of the data](#3)
1. [Analysis ](#4)
1. [Split train- and testset](#5)
1. [Model training](#6)
1. [Model validation](#7)
1. [Conclusion](#8)


<a id="1"></a> 

## 1. Import packages and load the data

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import sklearn.ensemble
import sklearn.model_selection
import sklearn.tree
import xgboost as xgb

In [ ]:
# Set plotly as panda's default way of plotting
pd.options.plotting.backend = "plotly"

In [ ]:
# Load data as a DataFrame
df = pd.read_csv("data/dataset.csv")

<a id="2"></a> 


## 2. Data Exploration
The first step of any Data Science project is to explore the dataset and familiarize yourself with the available variables.

Here is an overview with a short description of each variable:
| Variable | Description   |
|------|------|
| FAILURE  | Indicates whether the asset has failed. |
| YEAR_CONSTRUCTION | The year that asset was installed in the network. |
| YEAR_ACTIVE | The year in which the asset was first utilized. |
| AGE | The age of the asset (determined in 2018). |
| CABLE_COX1 | Cable type on installation side. |
| CABLE_COX2 | Cable type on the network side. |
| COX1==COX2 | Are cable 1 and 2 of the same type? |
| CONSTRUCTION_ORIG | Original construction type of the cable-joint (as registrated in the database systems) |
| CONSTRUCTION_EXP | Construction type estimated with a model |
| CONSTRUCTION_COX | Construction type estimated with another model |
| GROUND_TYPE | Type of ground/soil. 
| SUBSIDENCE  | Has the soil been subsided? |
| DEWATERING_DEPTH_CM | The difference in cm between ground water level and surface level. |

#### 2.1 First insights in the dataset!

In [ ]:
# First view of the data
df.head(10)

In [ ]:
# EXERCISE:
# Print the number of rows and columns.
print('nrow:', df.shape[0])
print('ncols:', df.shape[1])

In [ ]:
# EXERCISE:
# Show a short statistical summary (mean, standard deviation, etc) for the numeric values in the dataframe.
df.describe()

In [ ]:
# EXERCISE:
# Show data types for each column.
df.dtypes

#### 2.2 Analyse the variable we try to predict

In [ ]:
# EXERICSE:
# Make a count of the number of failures.
df["FAILURE"].value_counts()

#### 2.3 Analyse categorical variables

In [ ]:
# EXERCISE:
# Plot the columns that you want to inspect the values of.
categorical_columns = [
    "CABLE_COX1",
    "CABLE_COX2",
    "CONSTRUCTION_ORIG",
    "CONSTRUCTION_EXP",
    "CONSTRUCTION_COX",
    "GROUND_TYPE"
]

for col in categorical_columns:
    df.plot(
        kind='hist',
        x=col,
        title=col
    ).show()

#### 2.4 Analyse the distribution of the numerical variables

In [ ]:
# EXERICE:
# Now make some figures to analyse the distribution of the values in the numerical columns. 
# Experiment with different chart types. For inspiration see: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html
# Feel free to use explore an alternative library for producing figures if you feel like it.
df.plot(
    kind="hist",
    x="AGE"
)

In [ ]:
df.plot(
    kind="box",
    x="AGE"
)

#### 2.5 Check for missing data

When training a model the quality of your data can be a limiting factor.
Therefore, it is wise to check the completeness of the column in your dataset early on.

In [ ]:
# EXERCISE:
# Provide insights (table or figure) in the percentage of missing values per column.

In [ ]:
# Get insights in how many missing values (NULL or NANs) are in the dataset
df_missing = pd.DataFrame(
    data={
        'NUM_MISSING': df.isna().sum(axis='rows'),
        'NUM_TOTAL': len(df)
    }
)

# Compute percentage missing
df_missing['PCT_MISSING'] = df_missing['NUM_MISSING'] / df_missing['NUM_TOTAL']

# Sort dataframe based on
df_missing = df_missing.sort_values(by='PCT_MISSING', ascending=False)

# Show the 'missing' dataframe
df_missing

In [ ]:
# Visualize the results
df_missing.plot(
    kind='bar',
    y='PCT_MISSING',
    title="Missing Values"
)

<a id="3"></a> 


## 3. Data Preparation
In the data preparation step we want to re-structure our dataset to a format on which we can apply models.
And also remove factors (such as outliers, missing values, incorrect data, etc.) that negatively impact the ability to train a good model.

#### 3.1 Remove outliers

In [ ]:
# EXERCISE:
# Based on your results in the exploration step, try to remove some absurd values (outliers) that might negatively impact a machine learning model.
df = df[~(df["DEWATERING_DEPTH_CM"] <= -30)]
df = df[df["YEAR_CONSTRUCTION"] > 1900]

#### 3.2 Fill missing values

Not all models can handle handle missing values in input variables: NaN/None/etc.
In that case you have to come up with a strategy on how to replace missing values if you want to be able to use all variables.

In [ ]:
# EXERCISE:
# Fill in (or remove) the missing values in numeric columns. A good strategy might be to fill the missing with the statistical average.
df = df.fillna(df.mean(numeric_only=True))

#### 3.3 Apply one-hot-encoding on categorical variables
Most models expect categorical variables to be one-hot encoded.
See for more information: https://en.wikipedia.org/wiki/One-hot

In [ ]:
# EXERCISE:
# Find all the categorical variables and one-hot-encode them. 
# Hint: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
categorical_columns = [
    "CABLE_COX1",
    "CABLE_COX2",
    "CONSTRUCTION_ORIG",
    "CONSTRUCTION_EXP",
    "CONSTRUCTION_COX",
    "GROUND_TYPE"
]

df_prepped = pd.get_dummies(df, columns=categorical_columns)

# Check the prepped dataframe
df_prepped.head(10)

<a id="4"></a> 


## 4. Analyse

So far we have prepped the dataset and analysed the distributions of the variables.
Before we start to train some models, it is a good idea to analyse the different relationships between the variables. 
Most importantly, how they relate to the failure of cable joints.

In [ ]:
# EXERCISE: 
# Try make some useful figures to investigate the relationships between age-related variables and the amount of failures.
df_prepped.plot(
    kind="hist",
    x="AGE",
    color="FAILURE"
)

In [ ]:
# EXERCISE:
# Now try to find a way to visualise the relationships between the categorical variables and the failures of cable joints.

columns = [
    "COX1==COX2",
    "SUBSIDENCE"
]

plt.figure(figsize=(20, 140))
for col in columns:
    plt.figure()
    sns.barplot(
        x=df_prepped[col], 
        y=df_prepped["FAILURE"], 
        palette='Blues'
    )
    plt.show()

In [ ]:
# EXERCISE: 
# Compute a correlation matrix for all the variables in the dataset. Select subsets to have clear overview, everything in one plot becomes quite unreadable.
# What variables seem to be (strongly) correlated with "FAILURE" and with each other? Can you explain the relationships that you have found? And do they make sense?

In [ ]:
# Select the columns here
columns = ['FAILURE'] + df_prepped.columns[2:20].to_list()

# Compute correlation matrix
corrmat = df_prepped[columns].corr().round(2)

# Correlation matrix figure (with seaborn)
sns.set(rc={'figure.figsize':(16, 16)})
sns.heatmap(corrmat, vmax=.8, square=True, annot=True, cmap='RdBu_r')

<a id="5"></a> 


## 5. Split train- and testset
In order to validate how good you machine learning model is able to predict failures

In [ ]:
# EXERCISE: 
# Split the dataset into two dataframes one for training and the other for testing.
df_train, df_test = sklearn.model_selection.train_test_split(df_prepped, train_size=0.8)

<a id="6"></a> 
## 6. Model training
In this step we will train 3 different classification models.
You can use [Model Validation](#7) to evaluate the performance of the models.
Try to train and validate your models one at a time.

In [ ]:
# In this dictionary we will save the trained models
models = {}

In [ ]:
# Here we define our predictive variable
y_var = "FAILURE"
y_train = df_train[y_var]
y_test = df_test[y_var]

<a id="6a"></a> 
## 6.1 Decision Tree

For more information see: https://en.wikipedia.org/wiki/Decision_tree_learning

For the documentation of the sklearn implementation see: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [ ]:
# EXERCISE:
# Fit a Decision Tree model with the code below and experiment with different combinations of input variables 
# and hyperparameters, and investigate how they impact the results in the validation module below.

In [ ]:
# Select the input variables for the model
x_vars = [col for col in df_train.columns if col != y_var]

x_train = df_train.loc[:, x_vars]
x_test = df_test.loc[:, x_vars]

In [ ]:
# Train model
models["decision_tree"] = sklearn.tree.DecisionTreeClassifier(
    max_depth=16,
    min_samples_split=8,
    min_samples_leaf=2
).fit(x_train, y_train)

<a id="6b"></a> 
## 6.2 Random Forest 

For more information see: https://en.wikipedia.org/wiki/Random_forest

For the documentation of the sklearn implementation see: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
# EXERCISE:
# Fit a Random Forest model with the code below and experiment with different combinations of input variables 
# and hyperparameters, and investigate how they impact the results in the validation module below.

In [ ]:
# Select the input variables for the model
x_vars = [col for col in df_train.columns if col != y_var]

x_train = df_train.loc[:, x_vars]
x_test = df_test.loc[:, x_vars]

In [ ]:
models["random_forest"] = sklearn.ensemble.RandomForestClassifier(
    # Number of trees
    n_estimators=100,
    # Max depth of a tree
    max_depth=16,
    
).fit(x_train, y_train)

<a id="6v"></a> 
## 6.3 XGBoost
For more information see: https://en.wikipedia.org/wiki/XGBoost

For more information of the xgb implementation see: https://xgboost.readthedocs.io/en/stable/python/python_api.html

In [ ]:
# EXERCISE:
# Fit a XGBoost model with the code below and experiment with different combinations of input variables 
# and hyperparameters, and investigate how they impact the results in the validation module below.

In [ ]:
# Select the input variables for the model
x_vars = [col for col in df_train.columns if col != y_var]

x_train = df_train.loc[:, x_vars]
x_test = df_test.loc[:, x_vars]

In [ ]:
models["xgboost"] = xgb.XGBClassifier(
    random_state=42
).fit(x_train, y_train)

<a id="7"></a> 
## 7. Model Validation
Now that we have trained a single or multiple models, we want to know how good it can predict the failure of cable joints.

In [ ]:
# List fitted models
for model in models:
    print(model)

In [ ]:
# We select the model we want to evaluate here
clf = models["decision_tree"]

#### 7.1 Prediction on test-set

In [ ]:
# EXERCISE:
# Apply your model on the test-set and obtain two arrays:
# - Class predictions, that indicate what class the model predicts (True for failure, False for non-failure).
# - Class probabilities, that describe how certain the model is for each class (failure/non-failure).
# Hint: use help(clf) to see what functions are available.

In [ ]:
# Predict on the test set (True/False for Failure/Non-failure)
y_test_pred = clf.predict(x_test)

In [ ]:
# Predict probabilities as well
y_test_pred_proba = clf.predict_proba(x_test)

#### 7.2 Confusion Matrix

In [ ]:
# EXERCISE:
# Make a confusion matrix of the labels predicted by your model.

In [ ]:
# Confusion matrix
cm = sklearn.metrics.confusion_matrix(
    y_true=y_test,
    y_pred=y_test_pred
).T

In [ ]:
pd.DataFrame(cm).plot(
    title="Confusion Matrix",
    kind="imshow",
    x=["Negative", "Positive"],
    y=["Negative", "Positive"],
    labels={
        "x": "Actual Values",
        "y": "Predicted Values"
    },
    text_auto=True
)

#### 7.3 Accuracy

In [ ]:
# EXERCISE:
# Compute the accuracy of your model on the test-set.

In [ ]:
accuracy_score = sklearn.metrics.accuracy_score(y_test, y_test_pred)
print(accuracy_score)

#### 7.4 F1 score and PR curve 

In [ ]:
# EXERCISE:
# Compute the precision, recall and F1-score on the test predictions.
# What do these values tell you? Also try to make a precision/recall curve to illustrade the trade-off between the the two for different thresholds.

In [ ]:
# F1 score
f1_score = sklearn.metrics.f1_score(y_test, y_test_pred)

print("f1-score:", f1_score)

In [ ]:
# Compute Precision and Recall curve
precision, recall, _ = sklearn.metrics.precision_recall_curve(y_test,  y_test_pred_proba[:, 1])

# Visualize curve
(
    pd.DataFrame({'Precision': precision, 'Recall': recall})
    .plot(
        title='PR Curve',
        x='Recall', 
        y='Precision', 
        width=600, 
        height=600
    )
)

#### 7.5 AUC score and ROC Curve

In [ ]:
# EXERCISE:
# Now compute an ROC curve and AUC score and interpret them.

In [ ]:
# Compute ROC curve
fpr, tpr, t = sklearn.metrics.roc_curve(y_test, y_test_pred_proba[:, 1])

# Visualize ROC curve
(
    pd.DataFrame({'True Positive Rate': tpr, 'False Positive Rate': fpr})
    .plot(
        title='ROC Curve',
        x='False Positive Rate', 
        y='True Positive Rate', 
        width=600, 
        height=600
    )
)

In [ ]:
auc_score = sklearn.metrics.auc(fpr, tpr)
print("AUC score:", auc_score)

#### 7.6 Optimal Cut-off point

In [ ]:
# EXERCISE:
# The default threshold value that is applied on the predicted probabilty is often set at 0.5. 
# As you have seen with the PR and ROC curves, this threshold can be raised or lowered to prefer False Positives or False Negatives.
# Now imagine that the following business costs are implied if we would apply your model:
cost_tp = 50
cost_tn = 0
cost_fp = 50
cost_fn = 100

# Can you find the optimal threshold value for your model that minimizes the total cost of your predictions on the test-set?

In [ ]:
threshold_costs = []
for threshold in np.linspace(0, 1, 100):
    # Apply threshold
    _y_test_pred = y_test_pred_proba[:, 1] > threshold
    
    # Compute amount of TP, TN, FP and FN
    _cm = sklearn.metrics.confusion_matrix(_y_test_pred, y_test)
    tn = _cm[0][0]
    fn = _cm[0][1]
    fp = _cm[1][0]
    tp = _cm[1][1]
    
    # Compute costs
    total_cost = tp * cost_tp + tn * cost_tn + fp * cost_fp + fn * cost_fn
    
    threshold_costs.append({'threshold': threshold, 'total_cost': total_cost})
    
df_threshold_costs = pd.DataFrame(threshold_costs)

In [ ]:
df_threshold_costs.plot(
    x='threshold',
    y='total_cost'
)

In [ ]:
# Compute optimal threshold
df_threshold_costs.iloc[df_threshold_costs['total_cost'].argmin()]

#### 7.7 Feature Importance

In [ ]:
# EXERCISE:
# Could you obtain the most important features in your model?
# Is this in line with the observations you made in the data analysis step?

In [ ]:
# Feature importance
df_feature_importances = (
    pd.DataFrame(
        data={
            "FEATURE": clf.feature_names_in_,
            "IMPORTANCE": clf.feature_importances_
        }
    )
    .sort_values(
        by="IMPORTANCE", 
        ignore_index=True,
        ascending=False
    )
    .head(10)
)

df_feature_importances.plot(
    title="Feature Importances (Top 10)",
    kind="bar", 
    x="IMPORTANCE", 
    y="FEATURE",
)

<a id="8"></a> 
## 8. Conclusion

In [ ]:
# Imagine that you can inspect or replace 200 cable joints (in the test-set). Which ones would you choose? 
# How many failures would you miss and how many replacements were unnecessary?
# ...

In [ ]:
# Can you explain why some models seem to be able to predict failures better based on this dataset than others?
# ...